In [56]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### **원시데이터**

In [ ]:
train = pd.read_csv('/mnt/aix7206/data/train.csv')

In [ ]:
train.info()

In [ ]:
train.head(10)

In [ ]:
train.describe()

In [ ]:
train.groupby(['image_crops', 'image_fname'])['image_fname'].count()

### **생육지표가 라벨링되어 있는 데이터만 filtering**

In [7]:
train_label = train[train['growth_leaf_width'].notnull()]

In [8]:
train_label = pd.read_csv('/mnt/aix7206/data/train_label.csv')

In [ ]:
train_label.info()

In [ ]:
train_label.head()

In [ ]:
train_label.describe()

In [ ]:
train_label['image_crops'].value_counts()

In [ ]:
train_label.groupby('image_crops')['image_growth_stage'].value_counts()

In [ ]:
train_label['image_growth_stage'].value_counts()

In [ ]:
train_label['cat_name'].value_counts()

In [ ]:
train_label[train_label['image_growth_stage']=='정식기'][['growth_stem_length', 'growth_leaf_cnt', 'growth_leaf_width', 'growth_leaf_length', 'growth_leaf_index']].describe()

In [ ]:
train_label[train_label['image_growth_stage']=='생육기'][['growth_stem_length', 'growth_leaf_cnt', 'growth_leaf_width', 'growth_leaf_length', 'growth_leaf_index']].describe()

In [ ]:
train_label[train_label['image_growth_stage']=='수확기'][['growth_stem_length', 'growth_leaf_cnt', 'growth_leaf_width', 'growth_leaf_length', 'growth_leaf_index']].describe()

### **이미지 및 bounding box**

In [37]:
import selectivesearch
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import ast
%matplotlib inline

In [17]:
def find_file(root_folder, target_file):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        if target_file in filenames:
            return os.path.join(dirpath, target_file)
    return "파일을 찾을 수 없습니다."

In [18]:
# filter_leaf = train_label[train_label['cat_name']=='잎']
# len(filter_leaf)

In [19]:
# filter_leaf.to_csv('/mnt/aix7206/data/filter_leaf.csv', index=False)

In [20]:
filter_leaf = pd.read_csv('/mnt/aix7206/data/filter_leaf.csv')

In [21]:
# 사용 예시
root_directory = '/mnt/aix7206/data/' + filter_leaf['image_file_path'][1]  # 탐색을 시작할 루트 폴더 경로
file_name = filter_leaf['image_fname'][1]  # 찾고자 하는 파일명

file_path = find_file(root_directory, file_name)

In [ ]:
img = cv2.imread(file_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print('img shape:', img.shape)

plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

In [ ]:
# 바운딩 박스 좌표 변환
gt_box_list = ast.literal_eval(filter_leaf['ann_bbox'][1])
gt_box = list(map(int, gt_box_list))

# 'growth_leaf_length'와 'growth_leaf_width'의 1번째 행 데이터 추출
growth_leaf_length = filter_leaf['growth_leaf_length'][1]
growth_leaf_width = filter_leaf['growth_leaf_width'][1]

# 바운딩 박스 색상
red = (255, 0, 0)

# 이미지에 바운딩 박스 그리기
img_rgb = cv2.rectangle(img_rgb, (gt_box[0], gt_box[1]), (gt_box[0]+gt_box[2], gt_box[1]+gt_box[3]), color=red, thickness=2)

# 표시할 텍스트 (엽폭과 엽장의 정보를 문자열로 변환)
text = f'Length: {growth_leaf_length}, Width: {growth_leaf_width}'

# 텍스트 위치 지정 (바운딩 박스의 왼쪽 위 좌표를 기준으로)
text_position = (gt_box[0], gt_box[1] - 10)  # 바운딩 박스 위쪽에 텍스트 표시

# 이미지에 텍스트 그리기
cv2.putText(img_rgb, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# 이미지 출력
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.axis('off')  # 축을 숨김
plt.show()

In [ ]:
filter_whole = train_label[train_label['cat_name']=='주']
len(filter_whole)

In [32]:
filter_whole.to_csv('/mnt/aix7206/data/filter_whole.csv', index=False)

In [39]:
filter_whole = pd.read_csv('/mnt/aix7206/data/filter_whole.csv')

In [41]:
root_directory = '/mnt/aix7206/data/' + filter_whole['image_file_path'][1]  # 탐색을 시작할 루트 폴더 경로
file_name = filter_whole['image_fname'][1]  # 찾고자 하는 파일명

file_path = find_file(root_directory, file_name)

In [ ]:
img = cv2.imread(file_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print('img shape:', img.shape)

plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.show()

In [ ]:
# 바운딩 박스 좌표 변환
gt_box_list = ast.literal_eval(filter_whole['ann_bbox'][1])
gt_box = list(map(int, gt_box_list))

# 'growth_leaf_length'와 'growth_leaf_width'의 1번째 행 데이터 추출
growth_leaf_length = filter_whole['growth_leaf_length'][1]
growth_leaf_width = filter_whole['growth_leaf_width'][1]

# 바운딩 박스 색상
red = (255, 0, 0)

# 이미지에 바운딩 박스 그리기
img_rgb = cv2.rectangle(img_rgb, (gt_box[0], gt_box[1]), (gt_box[0]+gt_box[2], gt_box[1]+gt_box[3]), color=red, thickness=2)

# 표시할 텍스트 (엽폭과 엽장의 정보를 문자열로 변환)
text = f'Length: {growth_leaf_length}, Width: {growth_leaf_width}'

# 텍스트 위치 지정 (바운딩 박스의 왼쪽 위 좌표를 기준으로)
text_position = (gt_box[0], gt_box[1] - 10)  # 바운딩 박스 위쪽에 텍스트 표시

# 이미지에 텍스트 그리기
cv2.putText(img_rgb, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

# 이미지 출력
plt.figure(figsize=(8, 8))
plt.imshow(img_rgb)
plt.axis('off')  # 축을 숨김
plt.show()

In [ ]:
train = pd.read_csv('/mnt/aix7206/data/train.csv')

In [ ]:
len(train)

In [ ]:
len(train[train['cat_name']=='주'])

In [ ]:
len(train[train['cat_name']=='잎'])

In [ ]:
len(train[train['cat_name'].isnull()])

### **재배환경데이터 매칭(train만)**

ffill

In [7]:
train = pd.read_csv('/mnt/aix7206/data/train.csv')
val = pd.read_csv('/mnt/aix7206/data/val.csv')
env_train = pd.read_csv('/mnt/aix7206/data/train_aihub.csv')
env_val = pd.read_csv('/mnt/aix7206/data/val_aihub.csv')

In [29]:
train_ffill = train.fillna(method='ffill') 
train_ffill.to_csv('/mnt/aix7206/data/train_ffill.csv', index=False)

In [64]:
val_ffill = val.fillna(method='ffill')
val_ffill.to_csv('/mnt/aix7206/data/val_ffill.csv', index=False)

In [30]:
env_train_ffill = env_train.fillna(method='ffill') 
env_train_ffill.to_csv('/mnt/aix7206/data/train_aihub_ffill.csv', index=False)

In [31]:
env_val_ffill = env_val.fillna(method='ffill') 
env_val_ffill.to_csv('/mnt/aix7206/data/val_aihub_ffill.csv', index=False)

matching

In [ ]:
train_ffill = pd.read_csv('/mnt/aix7206/data/dump/train_ffill.csv')
env_train_ffill = pd.read_csv('/mnt/aix7206/data/dump/train_aihub_ffill.csv')

In [ ]:
print(len(train_ffill), len(env_train_ffill))

In [ ]:
train_ffill.head()

In [ ]:
env_train_ffill.head()

In [ ]:
# 'env'로 시작하는 열 필터링
env_columns = [col for col in env_train_ffill.columns if col.startswith('env')]

# 'image_date_captured'와 'ann_bbox' 기준으로 두 데이터프레임을 병합
merged_df_train = pd.merge(
    train_ffill,
    env_train_ffill[['image_date_captured', 'ann_bbox', 'ann_area', 'ann_segmentation', 'ann_image_id', 'image_create_date', 'ann_id'] + env_columns], 
    on=['image_date_captured', 'ann_bbox', 'ann_area', 'ann_segmentation', 'ann_image_id', 'image_create_date', 'ann_id'], 
    how='left'
)

len(merged_df_train)

In [ ]:
merged_df_train.info()

In [29]:
merged_df_train.to_csv('train_env.csv', index=False)

### **bbox 수정**

In [ ]:
train_env = pd.read_csv('/mnt/aix7206/data/dump/train_env.csv')

In [ ]:
train_env.info()

In [ ]:
train_env.head()

In [8]:
def modify_bbox(bbox):
    # 소수점 제거
    bbox = ast.literal_eval(bbox)
    bbox = [int(value) for value in bbox]
    # bbox 수정
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]
    
    return bbox

train_env['ann_bbox_new'] = train_env['ann_bbox'].apply(modify_bbox)


In [ ]:
train_env.head()

In [10]:
train_env.to_csv('train_final.csv', index=False)

### 
---

### ***2024.09.26 data updates***

In [3]:
import os

train_label = '/mnt/aix7206/data/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/1.Training/라벨링데이터'
train_raw = '/mnt/aix7206/data/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/1.Training/원천데이터'
val_label = '/mnt/aix7206/data/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/2.Validation/라벨링데이터'
val_raw = '/mnt/aix7206/data/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/2.Validation/원천데이터'

curr_train_label = '/mnt/aix7206/data/final/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/1.Training/라벨링데이터'
curr_train_raw = '/mnt/aix7206/data/final/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/1.Training/원천데이터'
curr_val_label = '/mnt/aix7206/data/final/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/2.Validation/라벨링데이터'
curr_val_raw = '/mnt/aix7206/data/final/100.지능형 수직농장 통합 데이터(엽채류)/01.데이터/2.Validation/원천데이터'

def file_cnt(directory):
    file_count = 0
    for root, dirs, files in os.walk(directory):
        file_count += len(files)
    return file_count

def file_cnt_nodir(directory):
    file_count = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
    return file_count


In [ ]:
print(f"이전 train_label 개수:{file_cnt(train_label)}ea,\
      \n이전 train_raw 개수:{file_cnt(train_raw)}ea,\
      \n이전 val_label 개수: {file_cnt(val_label)}ea, \
      \n이전 val_raw 개수: {file_cnt(val_raw)}ea")

print(f"new train_label 개수:{file_cnt_nodir(curr_train_label)}ea,\
    \nnew train_raw 개수:{file_cnt_nodir(curr_train_raw)}ea,\
    \nnew val_label 개수:{file_cnt_nodir(curr_val_label)}ea, \
    \nnew val_raw 개수: {file_cnt_nodir(curr_val_raw)}ea")

### ***2024.09.30 > 10개의 폴더에 랜덤하게 배분 후 json to csv***

In [ ]:
import os
import random
import shutil

def create_folders(base_dir, num_folders=10):
    # 10개의 폴더 생성
    folder_paths = []
    for i in range(1, num_folders + 1):
        folder_path = os.path.join(base_dir, f'folder_{i}')
        os.makedirs(folder_path, exist_ok=True)  # 이미 존재하는 폴더가 있을 경우 무시
        folder_paths.append(folder_path)
    return folder_paths

def distribute_files(base_dir, num_folders=10):
    # 파일을 분배할 10개의 폴더 생성
    folders = create_folders(base_dir, num_folders)
    
    # 하위 폴더의 모든 파일 탐색
    all_files = []
    for root, _, files in os.walk(base_dir):
        if root not in folders:  # 생성한 폴더 안의 파일들은 제외
            for file in files:
                file_path = os.path.join(root, file)
                all_files.append(file_path)
    
    # 파일을 무작위로 섞기
    random.shuffle(all_files)

    # 파일을 10개의 폴더에 무작위로 분배
    for i, file_path in enumerate(all_files):
        target_folder = folders[i % num_folders]  # 순차적으로 폴더 선택
        shutil.move(file_path, os.path.join(target_folder, os.path.basename(file_path)))
        print(f"Moved {file_path} to {target_folder}")

# 탐색할 상위 디렉토리 경로 설정
base_directory = '/mnt/aix7206/data/final/라벨링데이터'

distribute_files(base_directory)


In [17]:
def count_files_in_subfolders(parent_folder):
    total = 0
    # 하위 폴더 탐색
    for root, dirs, files in os.walk(parent_folder):
        if root != parent_folder:  # 상위 폴더 제외
            folder_name = os.path.basename(root)
            print(f"Folder: {folder_name} - Number of files: {len(files)}")
            total += len(files)
    print(f"Total number of files: {total}")

In [ ]:
parent_folder = "/mnt/aix7206/data/final/라벨링데이터"
count_files_in_subfolders(parent_folder)

In [3]:
import os
import json
import pandas as pd
import time
from tqdm import tqdm

def combine_json_files_recursive(directory, output_csv_file):
    start_time = time.time()  # 시작 시간 기록
    combined_df = pd.DataFrame()

    # 모든 JSON 파일 수 계산
    total_files = sum([len(files) for _, _, files in os.walk(directory) if any(file.endswith('.json') for file in files)])

    # tqdm을 사용하여 진행률 표시
    with tqdm(total=total_files, desc="Processing JSON files", unit="file") as pbar:
        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith('.json'):
                    file_path = os.path.join(root, file)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)

                        # 필요한 데이터 추출
                        image_info = data.get('images', {})
                        environments = data.get('environments', {})
                        growth_index = data.get('growth_index', {})
                        etc_infor = data.get('etc_infor', {})
                        annotations = data.get('annotations', {})
                        licenses = data.get('licenses', {})
                        categories = data.get('categories', {})
                        
                        # 데이터 프레임 생성 (비어 있는 값도 그대로 null로 저장)
                        image_df = pd.json_normalize(image_info).add_prefix('image_') if image_info else pd.DataFrame([None])
                        env_df = pd.json_normalize(environments).add_prefix('env_') if environments else pd.DataFrame([None])
                        growth_df = pd.json_normalize(growth_index).add_prefix('growth_') if growth_index else pd.DataFrame([None])
                        etc_df = pd.json_normalize(etc_infor).add_prefix('etc_') if etc_infor else pd.DataFrame([None])
                        ann_df = pd.json_normalize(annotations).add_prefix('ann_') if annotations else pd.DataFrame([None])
                        lic_df = pd.json_normalize(licenses).add_prefix('lic_') if licenses else pd.DataFrame([None])
                        cat_df = pd.json_normalize(categories).add_prefix('cat_') if categories else pd.DataFrame([None])

                        # 데이터 프레임 병합
                        merged_df = pd.concat([image_df, env_df, growth_df, etc_df, ann_df, lic_df, cat_df], axis=1)
                        
                        # 전체 데이터 프레임에 추가
                        combined_df = pd.concat([combined_df, merged_df], ignore_index=True)

                    pbar.update(1)  # 진행 상태 업데이트

    # 완료된 DataFrame을 CSV로 저장 (null 값은 그대로 포함)
    combined_df.to_csv(output_csv_file, index=False, encoding='utf-8-sig', na_rep='null')

    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time  # 걸린 시간 계산
    print(f"Time taken: {elapsed_time:.2f} seconds")  # 걸린 시간 출력

In [ ]:
for i in range(1, 11):
    print(f"folder_{i} start")
    # 사용 예시
    directory = f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}'  # JSON 파일들이 있는 폴더 경로로 변경하세요
    output_csv_file = f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv'  # 결과로 저장할 CSV 파일 이름
    combine_json_files_recursive(directory, output_csv_file)
    print(f"folder_{i} end")

***데이터 개수 확인***

In [8]:
total = 0
for i in range(1, 11):
    df = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')
    print(f"folder_{i} 개수: {len(df)}ea / 중복개수: {df.duplicated().sum()}ea")
    total += len(df)
print(f"전체 데이터 개수: {total}ea")

folder_1 개수: 52180ea / 중복개수: 0ea


/tmp/ipykernel_2981925/1736039679.py:3: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')


folder_2 개수: 52283ea / 중복개수: 0ea
folder_3 개수: 52349ea / 중복개수: 0ea


/tmp/ipykernel_2981925/1736039679.py:3: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')


folder_4 개수: 52300ea / 중복개수: 0ea
folder_5 개수: 52540ea / 중복개수: 0ea
folder_6 개수: 52513ea / 중복개수: 0ea
folder_7 개수: 52537ea / 중복개수: 0ea
folder_8 개수: 51984ea / 중복개수: 0ea


/tmp/ipykernel_2981925/1736039679.py:3: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')


folder_9 개수: 52375ea / 중복개수: 0ea
folder_10 개수: 52178ea / 중복개수: 0ea
전체 데이터 개수: 523239ea


/tmp/ipykernel_2981925/1736039679.py:3: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')


### **재배환경데이터 매칭(train만) + bbox 수정**

In [49]:
import pandas as pd
import os
import random
import shutil
import time
import ast

In [50]:
##### bbox 수정 #####
def modify_bbox(bbox):
    # 소수점 제거
    bbox = ast.literal_eval(bbox)
    bbox = [int(value) for value in bbox]
    # bbox 수정
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]
    
    return bbox

##### 재배환경데이터 매칭 #####
env_train = pd.read_csv('/mnt/aix7206/data/dump/train_aihub_ffill.csv')
env = env_train.drop_duplicates()

def process_folder(i):
    print(f'folder_{i} start')
    
    # CSV 파일 읽기
    train = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')
    
    # 결측값 채우기
    train_ffill = train.fillna(method='ffill')
    print(f"folder_{i} merge 전: {len(train_ffill)}ea")
    
    # 'env'로 시작하는 열 필터링
    env_columns = [col for col in env.columns if col.startswith('env')]
    
    # 'image_date_captured'와 'ann_bbox' 기준으로 두 데이터프레임 병합
    merged_df_train = pd.merge(
        train_ffill,
        env[['image_image_id', 'growth_stem_length', 'growth_leaf_cnt', 'growth_leaf_width', 'growth_leaf_length', 'growth_stem_thick', 'growth_fr_weight',  'image_date_captured', 'ann_bbox', 'ann_area', 'ann_segmentation', 'ann_image_id', 'image_create_date', 'ann_id', 'cat_id', 'ann_category_id'] + env_columns], 
        on=['image_image_id', 'growth_stem_length', 'growth_leaf_cnt', 'growth_leaf_width', 'growth_leaf_length', 'growth_stem_thick', 'growth_fr_weight',  'image_date_captured', 'ann_bbox', 'ann_area', 'ann_segmentation', 'ann_image_id', 'image_create_date', 'ann_id', 'cat_id', 'ann_category_id'], 
        how='left'
    )
    print(f"folder_{i} merge 후: {len(merged_df_train)}ea")
    
    # bbox 수정
    merged_df_train['ann_bbox_new'] = merged_df_train['ann_bbox'].apply(modify_bbox)
    
    # 결과를 새로운 CSV 파일로 저장
    merged_df_train.to_csv(f'/mnt/aix7206/data/final/라벨링데이터/train_{i}_final.csv', index=False)
    
    print(f'folder_{i} end')

def merge_final_files(num_folders, output_file):
    # train_{i}_final 파일들을 하나의 데이터프레임으로 병합
    combined_df = pd.DataFrame()
    
    for i in range(1, num_folders + 1):
        file_path = f'/mnt/aix7206/data/final/라벨링데이터/train_{i}_final.csv'
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # 최종 병합된 데이터를 train_final.csv로 저장
    combined_df.to_csv(output_file, index=False)
    print(f"All files merged into {output_file}")

def main():
    num_folders = 10
    start_time = time.time()  # 시작 시간 기록
    
    # 각 폴더 처리
    for i in range(1, num_folders + 1):
        folder_start_time = time.time()  # 각 폴더 처리 시작 시간
        
        process_folder(i)
        
        # 각 폴더 처리에 걸린 시간 계산
        folder_elapsed_time = time.time() - folder_start_time
        total_elapsed_time = time.time() - start_time
        
        # 남은 폴더 계산
        remaining_folders = num_folders - i
        
        # 예상 남은 시간 계산
        if i > 0:
            avg_time_per_folder = total_elapsed_time / i
            estimated_remaining_time = avg_time_per_folder * remaining_folders
            print(f"Estimated time remaining: {estimated_remaining_time:.2f} seconds")
    
    # 총 소요 시간
    total_time = time.time() - start_time
    print(f"All folders processed. Total time: {total_time:.2f} seconds")
    
    # 모든 train_{i}_final.csv 파일들을 하나로 병합
    merge_final_files(num_folders, '/mnt/aix7206/data/final/라벨링데이터/train_final.csv')

/tmp/ipykernel_3544045/2923626232.py:13: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  env_train = pd.read_csv('/mnt/aix7206/data/dump/train_aihub_ffill.csv')


In [51]:
# 메인 함수 실행
if __name__ == "__main__":
    main()

folder_1 start


/tmp/ipykernel_3544045/2923626232.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ffill = train.fillna(method='ffill')


folder_1 merge 전: 52180ea
folder_1 merge 후: 52180ea
folder_1 end
Estimated time remaining: 101.73 seconds
folder_2 start


/tmp/ipykernel_3544045/2923626232.py:20: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')
/tmp/ipykernel_3544045/2923626232.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ffill = train.fillna(method='ffill')


folder_2 merge 전: 52283ea
folder_2 merge 후: 52283ea
folder_2 end
Estimated time remaining: 92.55 seconds
folder_3 start


...

folder_9 merge 전: 52375ea
folder_9 merge 후: 52375ea
folder_9 end
Estimated time remaining: 11.56 seconds
folder_10 start


/tmp/ipykernel_3544045/2923626232.py:20: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')
/tmp/ipykernel_3544045/2923626232.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_ffill = train.fillna(method='ffill')


folder_10 merge 전: 52178ea
folder_10 merge 후: 52178ea
folder_10 end
Estimated time remaining: 0.00 seconds
All folders processed. Total time: 118.84 seconds


/tmp/ipykernel_3544045/2923626232.py:53: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3544045/2923626232.py:53: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3544045/2923626232.py:53: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_3544045/2923626232.py:53: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


All files merged into /mnt/aix7206/data/final/라벨링데이터/train_final.csv


***데이터 개수 비교***

In [57]:
total = 0
for i in range(1, 11):
    df = pd.read_csv(f'/mnt/aix7206/data/final/라벨링데이터/folder_{i}_new.csv')
    total += len(df)
print(f"merge 전 데이터 개수: {total}ea")

final = pd.read_csv('/mnt/aix7206/data/final/라벨링데이터/train_final.csv')
print(f"merge 후 데이터 개수: {len(final)}ea")

merge 전 데이터 개수: 523239ea
merge 후 데이터 개수: 523239ea


***val dataset bbox 수정 + train & val 칼럼 매칭***

In [81]:
final = pd.read_csv('/mnt/aix7206/data/train.csv')
val_final = pd.read_csv('/mnt/aix7206/data/val.csv')
print(set(final.columns).difference(set(val_final.columns)))
print(set(val_final.columns).difference(set(final.columns)))

{'ann_bbox_new', '0'}
set()


In [82]:
final.drop(["0"], axis=1, inplace=True)
val_final['ann_bbox_new'] = val_final['ann_bbox'].apply(lambda x: modify_bbox(x) if pd.notna(x) else x)

In [83]:
print(set(final.columns).difference(set(val_final.columns)))
print(set(val_final.columns).difference(set(final.columns)))

set()
set()


In [84]:
final.to_csv('/mnt/aix7206/data/train.csv', index=False)
val_final.to_csv('/mnt/aix7206/data/val.csv', index=False)